In [39]:
import pandas as pd
import sqlite3
import json
import os
from glob import glob
from datetime import datetime

In [40]:
files = [
    "./intermediate_results/v1_consolidato_2018_2025.csv",
    "./intermediate_results/v2_consolidato_2022_2024.csv",
    "./intermediate_results/v2b_consolidato_2025.csv"
]

# Lettura e allineamento colonne
dfs = [pd.read_csv(f, sep=";", low_memory=False) for f in files]
all_cols = sorted(set().union(*(df.columns for df in dfs)))
df = pd.concat([df.reindex(columns=all_cols) for df in dfs], ignore_index=True)

# Rimuove l'indice salvato e applica il filtro
print(len(df))
df = df.loc[:, ~df.columns.str.startswith('Unnamed')]
df = df[~((df['totale'] == 0) & (df['esito'] != 'NON AMMESSO'))]
print(len(df))

882991
882837


In [41]:
# df.to_csv("out.csv", sep=";")

In [42]:
df_names = pd.read_csv('./course-equivalence.csv', sep=";")
df_names

,code,name,name1,name2,name3,name4,name5
0,CEE1T1,CIVIL AND ENVIRONMENTAL ENGINEERING (L-7),CIVIL AND ENVIRONMENTAL ENGINEERING,NaN,NaN,NaN,NaN
1,ECE1T3,ELECTRONIC AND COMMUNICATIONS ENGINEERING (ING...,ELECTRONIC AND COMMUNICATIONS ENGINEERING (ING...,ELECTRONIC AND COMMUNICATIONS ENGINEERING,NaN,NaN,NaN
2,AER1T1,INGEGNERIA AEROSPAZIALE (L-9),INGEGNERIA AEROSPAZIALE,NaN,NaN,NaN,NaN
3,BIO1T1,INGEGNERIA BIOMEDICA (L-9),INGEGNERIA BIOMEDICA,NaN,NaN,NaN,NaN
4,CHI1T1,INGEGNERIA CHIMICA E ALIMENTARE (L-9),INGEGNERIA CHIMICA E ALIMENTARE,NaN,NaN,NaN,NaN
5,CIV1T1,INGEGNERIA CIVILE (L-7),INGEGNERIA CIVILE,NaN,NaN,NaN,NaN
6,MAT1T1,INGEGNERIA DEI MATERIALI (L-9),INGEGNERIA DEI MATERIALI,NaN,NaN,NaN,NaN
7,CIN1T3,INGEGNERIA DEL CINEMA E DEI MEZZI DI COMUNICAZ...,INGEGNERIA DEL CINEMA E DEI MEZZI DI COMUNICAZ...,NaN,NaN,NaN,NaN
8,AUT1T1,INGEGNERIA DELL'AUTOVEICOLO (L-9); INGEGNERIA ...,INGEGNERIA DELL'AUTOVEICOLO (AUTOMOTIVE ENGINE...,INGEGNERIA DELL'AUTOVEICOLO,NaN,NaN,NaN
9,PRO1A1,INGEGNERIA DELLA PRODUZIONE INDUSTRIALE (L-9) ...,INGEGNERIA DELLA PRODUZIONE INDUSTRIALE (Athlone),INGEGNERIA DELLA PRODUZIONE INDUSTRIALE,NaN,NaN,NaN


In [43]:
# read course equivalences
df_names = pd.read_csv('./course-equivalence.csv', sep=";")
modern_name = {}
modern_name_list = {}

for i, row in df_names.iterrows():
    if isinstance(row['name'], str) and len(row['name']) > 3:
        modern_name_list[row['code']] = row['name']
        modern_name[row['name']] = row['code']
        for j in range(1,6):
            alt_name = row[f'name{j}']
            if isinstance(alt_name, str) and len(alt_name) > 3:
                modern_name[alt_name] = row['code']
            
with open('datasets/courses.json', 'w') as outfile:
    json.dump(modern_name_list, outfile)

In [44]:
# df.to_csv("test.csv", sep=";")

In [45]:
debug = None
def rimpiazza_nome(old_name):
    global debug
    
    debug = old_name
    if old_name == "Ingegneria":
        return old_name
    if not isinstance(old_name, str):
        return old_name
    return modern_name[old_name]

# replace course specif
# rimpiazza i nomi dei corsi "strani" con un nome canonico
for c in ["course", "corso di immatricolazione", "corso unico", "corso assegnato", "corso prenotato"]:
    df[c] = df[c].apply(lambda x: rimpiazza_nome(x))
    # df[c] = df[c].str.replace(" \(.+","", regex=True).str.strip()

# set superset uppercase for consistency with other courses
# i supercorsi sono DESIGN, INGEGNERIA, ARCHITETTURA, PIANIFICAZIONE
df['supercorso'] = df['course'].str.upper()
del df['course']

In [46]:
# esporto il tutto in formato sqlite, idoneo a fare estrazioni
sqlitefile = f"./intermediate_results/tilstat_{str(datetime.now()).replace(':','-')}.sqlite"
conn = sqlite3.connect(sqlitefile)
df.to_sql('tilstat', conn, if_exists='replace', index=False)

882837

In [47]:
sqlitefile = sorted(glob(f'./intermediate_results/tilstat_*.sqlite'), key=os.path.getmtime, reverse=True)[0]
conn = sqlite3.connect(sqlitefile)

In [48]:
# PARANOID CHECKS (ogni riga ha almeno un corso tra imma, unico e assegnato)

cur = conn.cursor()
cur.execute("""
    SELECT
        ("corso di immatricolazione" IS NOT NULL +
         "corso unico" IS NOT NULL +
         "corso assegnato" IS NOT NULL) as chk,
         count(*)
    FROM tilstat
    GROUP BY
        chk 
""")
assert cur.fetchall() == [(1,len(df))]

In [49]:
# TIPOLOGIE ESITO

cur = conn.cursor()
cur.execute("""
    SELECT
        distinct(esito)
    FROM tilstat
    
""")
#list(map(lambda x: x[0], cur.description)),
cur.fetchall()

[('ASSEGNATO',),
 ('LISTA DI ATTESA',),
 ('NON AMMESSO',),
 ("RINUNCIA ALL'IMMATRICOLAZIONE",),
 ('IMMATRICOLATO',),
 ('RINUNCIA DOPO IMMA',),
 ('RINUNCIA ALLA LISTA DI ATTESA',),
 (None,),
 ('NON AMMESSO  - IMMATRICOLAZIONI CHIUSE AI SENSI DELL’ART 8 PUNTO 6 del D.R. N. 122/2018',),
 ('NON AMMESSO - IMMATRICOLAZIONI CHIUSE AI SENSI DEL D.R. N. 1112/2019',),
 ('AMMESSA AD ANNI SUCCESSIVI AL PRIMO',),
 ('NON AMMESSO - GRADUATORIE CHIUSE',),
 ('NON AMMESSO: REQUISITI LINGUISTICI NON RISPETTATI',),
 ('PRENOTATO',),
 ('ASSEGNATO (**)',),
 ('PRENOTATO (**)',),
 ('NON AMMESSO  - IMMATRICOLAZIONI CHIUSE AI SENSI DELL’ART 9 PUNTO 4 del D.R. N. 121/2018',),
 ('NON AMMESSO - IMMATRICOLAZIONI CHIUSE AI SENSI DELL’ART 9 PUNTO 4 del D.R. N. 121/2018',),
 ('RINUNICA DOPO IMMA',),
 ('ANNULLAMENTO IMMATRICOLAZIONE',),
 ('IMMATRICOLAZIONE ANNULLATA',),
 ('RINUNCIA DOPO IMMATRICOLAZIONE',)]

In [50]:
# ANNI

cur = conn.cursor()
cur.execute("""
    SELECT DISTINCT year FROM tilstat
    
""")
#list(map(lambda x: x[0], cur.description)),
print(cur.fetchall())

cur = conn.cursor()
cur.execute("""
    SELECT DISTINCT year FROM (SELECT
        CAST(substr(date,1,4) as INTEGER) as year
    FROM tilstat)
    
""")
#list(map(lambda x: x[0], cur.description)),
print(cur.fetchall())

[(2018,), (2019,), (2020,), (2021,), (2022,), (2023,), (2024,), (2025,)]
[(2017,), (2018,), (2019,), (2020,), (2021,), (2022,), (2023,), (2024,), (7,), (14,), (21,), (24,), (29,), (4,), (12,), (19,), (26,), (6,), (13,), (20,), (23,), (28,), (3,), (11,), (25,), (5,), (10,), (17,)]


In [51]:
# TILSTAT

cur = conn.cursor()

cur.execute(f"""
    SELECT
        COALESCE("corso unico", "corso assegnato", "corso prenotato", "supercorso") as corso,
        
        /* grad_n <=> specific date in a year */
        CAST(substr(date,9,2) as INTEGER) / 8 as week,
        substr(date,6,2) as month,
        year,
        
        MIN(grad_n) as first_scorr,
        MAX(grad_n) as last_scorr,
        COUNT(*) as people_n,
        MIN(totale) as min_tot,
        
        COALESCE("user", "af user") as matr,
        
        /* following star is a sqlite magic: can return all line of MIN value, to debug and stats purpose */ 
        *
    FROM
        tilstat as ts1
    WHERE
        /* select only taken students */
        ("esito" LIKE "ASSEGNATO" OR
        "esito" LIKE "PRENOTATO" OR
        "esito" LIKE "IMMATRICOLATO") AND
        
        /* contig or not */
        conting = 0 AND
        
        instr(matr, "*") = 0
    GROUP BY
        corso, year, month, week
    HAVING
        /* hide super-corso of alredy shown course */
        corso <> "INGEGNERIA"
    
    /* this order prevent inversions in view */
    ORDER BY
        month, week
    
""")
headers = list(map(lambda x: x[0], cur.description))
out = [{h:v for h,v in zip(headers, row)} for row in cur.fetchall()]
with open('datasets/tilstat.json', 'w') as outfile:
    json.dump(out, outfile)

In [58]:
import shutil
import subprocess

# Percorsi dei file da copiare
source_tilstat = 'datasets/tilstat.json'
source_courses = 'datasets/courses.json'

# Percorsi di destinazione
dest_tilstat = './../../webapp/src/data/tilstat.json'
dest_courses = './../../webapp/src/data/courses.json'

# Copia i file
shutil.copyfile(source_tilstat, dest_tilstat)
print(f"Copiato {source_tilstat} in {dest_tilstat}")

shutil.copyfile(source_courses, dest_courses)
print(f"Copiato {source_courses} in {dest_courses}")

# # Esegui il comando npm build
# try:
#     subprocess.run(['npm', 'run', 'build'], cwd='./../../webapp', check=True)
#     print("Build completata con successo.")
# except subprocess.CalledProcessError as e:
#     print(f"Errore durante la build: {e}")

Copiato datasets/tilstat.json in ./../../webapp/src/data/tilstat.json
Copiato datasets/courses.json in ./../../webapp/src/data/courses.json


FileNotFoundError: [WinError 2] Impossibile trovare il file specificato

In [52]:
# TILSTAT

cur = conn.cursor()

cur.execute(f"""
    SELECT corso, year, COUNT(distinct matr) as cnt FROM
    
    (SELECT
        year,
        COALESCE("corso unico", "corso assegnato", "corso prenotato", "supercorso") as corso,
        COALESCE("user", "af user") as matr
    FROM
        tilstat as ts1)
        
    GROUP BY corso, year
    HAVING
        /* hide super-corso of alredy shown course */
    corso <> "INGEGNERIA"
""")
headers = list(map(lambda x: x[0], cur.description))
out = [{h:v for h,v in zip(headers, row)} for row in cur.fetchall()]
for row in out:
    print(modern_name_list[row['corso']],"\t", row['year'], "\t", row['cnt'])

INGEGNERIA AEROSPAZIALE (L-9) 	 2018 	 1
INGEGNERIA AEROSPAZIALE (L-9) 	 2019 	 1
INGEGNERIA AEROSPAZIALE (L-9) 	 2020 	 1
INGEGNERIA AEROSPAZIALE (L-9) 	 2021 	 1
INGEGNERIA AEROSPAZIALE (L-9) 	 2022 	 1
INGEGNERIA AEROSPAZIALE (L-9) 	 2023 	 1
INGEGNERIA AEROSPAZIALE (L-9) 	 2024 	 1
INGEGNERIA AEROSPAZIALE (L-9) 	 2025 	 1
INGEGNERIA PER L'AMBIENTE E IL TERRITORIO (L-7) 	 2018 	 1
INGEGNERIA PER L'AMBIENTE E IL TERRITORIO (L-7) 	 2019 	 1
INGEGNERIA PER L'AMBIENTE E IL TERRITORIO (L-7) 	 2020 	 1
INGEGNERIA PER L'AMBIENTE E IL TERRITORIO (L-7) 	 2021 	 1
INGEGNERIA PER L'AMBIENTE E IL TERRITORIO (L-7) 	 2022 	 1
INGEGNERIA PER L'AMBIENTE E IL TERRITORIO (L-7) 	 2023 	 1
INGEGNERIA PER L'AMBIENTE E IL TERRITORIO (L-7) 	 2024 	 1
INGEGNERIA PER L'AMBIENTE E IL TERRITORIO (L-7) 	 2025 	 1
ARCHITETTURA (ARCHITECTURE) 	 2021 	 1
ARCHITETTURA (ARCHITECTURE) 	 2022 	 1
ARCHITETTURA (ARCHITECTURE) 	 2023 	 1
ARCHITETTURA (ARCHITECTURE) 	 2024 	 1
ARCHITETTURA (ARCHITECTURE) 	 2025 	 1
INGEG

In [53]:
# TILSTAT

cur = conn.cursor()

cur.execute(f"""
    SELECT year, COUNT(distinct matr) as cnt FROM
    
    (SELECT
        year,
        COALESCE("user", "af user") as matr
    FROM
        tilstat as ts1)
        
    GROUP BY year
""")
headers = list(map(lambda x: x[0], cur.description))
out = [{h:v for h,v in zip(headers, row)} for row in cur.fetchall()]
out

[{'year': 2018, 'cnt': 1},
 {'year': 2019, 'cnt': 1},
 {'year': 2020, 'cnt': 1},
 {'year': 2021, 'cnt': 1},
 {'year': 2022, 'cnt': 1},
 {'year': 2023, 'cnt': 1},
 {'year': 2024, 'cnt': 1},
 {'year': 2025, 'cnt': 1}]

In [54]:
out

[{'year': 2018, 'cnt': 1},
 {'year': 2019, 'cnt': 1},
 {'year': 2020, 'cnt': 1},
 {'year': 2021, 'cnt': 1},
 {'year': 2022, 'cnt': 1},
 {'year': 2023, 'cnt': 1},
 {'year': 2024, 'cnt': 1},
 {'year': 2025, 'cnt': 1}]

In [55]:
# TILSTAT

cur = conn.cursor()

cur.execute(f"""
    SELECT
        DISTINCT( COALESCE(
            "corso unico",
            "corso assegnato",
            "corso prenotato",
            "corso di immatricolazione")) as corso
    FROM tilstat
    
""")
headers = list(map(lambda x: x[0], cur.description))
courses = [c[0] for c in cur.fetchall() if c[0] is not None]
courses


['ARCHITETTURA',
 'AMB1T1',
 'MTM1T1',
 'INF1T3',
 'BIO1T1',
 'GES1T4',
 'AER1T1',
 'ELN1T3',
 'CHI1T1',
 'MEC1T1',
 'PRO1B1',
 'EDI1T1',
 'CIN1T3',
 'MAT1T1',
 'ENE1T1',
 'AUT1T1',
 'ECE1T3',
 'FIS1T3',
 'PRO1A1',
 'PRO1N1',
 'ELT1T1',
 'CIV1T1',
 'CEE1T1',
 'TIM1T4']

In [57]:
# TILSTAT HOF

out = []

for c in courses:
    cur = conn.cursor()
    cur.execute(f"""
        SELECT
            COALESCE(
                "corso/i in lista di attesa",
                "corsi in lista di attesa",
                "corso unico",
                "corso assegnato",
                "corso prenotato",
                "supercorso"
            ) as corso,
            COALESCE("user", "af user") as matr,
            year,
            COUNT(*) as count
        FROM
            tilstat
        WHERE      
            /* contig or not */
            conting = 0 AND

            /* remove special matr */
            instr(matr, "*") = 0 AND

            grad_n = 0 AND
            corso LIKE "%{c}%"

        GROUP BY
            year

    """)
    headers = list(map(lambda x: x[0], cur.description))
    out += [{h:v for h,v in zip(headers, row)} for row in cur.fetchall()]
    
with open('datasets/tilstathof.json', 'w') as outfile:
    json.dump(out, outfile)
out

[{'corso': 'AMB1T1', 'matr': 'user', 'year': 2022, 'count': 395},
 {'corso': 'AMB1T1', 'matr': 'user', 'year': 2023, 'count': 287},
 {'corso': 'AMB1T1', 'matr': 'user', 'year': 2024, 'count': 303},
 {'corso': 'AMB1T1', 'matr': 'user', 'year': 2025, 'count': 179},
 {'corso': 'MTM1T1', 'matr': 'user', 'year': 2022, 'count': 357},
 {'corso': 'MTM1T1', 'matr': 'user', 'year': 2023, 'count': 316},
 {'corso': 'MTM1T1', 'matr': 'user', 'year': 2024, 'count': 355},
 {'corso': 'MTM1T1', 'matr': 'user', 'year': 2025, 'count': 311},
 {'corso': 'INF1T3', 'matr': 'user', 'year': 2022, 'count': 1497},
 {'corso': 'INF1T3', 'matr': 'user', 'year': 2023, 'count': 1319},
 {'corso': 'INF1T3', 'matr': 'user', 'year': 2024, 'count': 1306},
 {'corso': 'INF1T3', 'matr': 'user', 'year': 2025, 'count': 1276},
 {'corso': 'BIO1T1', 'matr': 'user', 'year': 2022, 'count': 1442},
 {'corso': 'BIO1T1', 'matr': 'user', 'year': 2023, 'count': 1101},
 {'corso': 'BIO1T1', 'matr': 'user', 'year': 2024, 'count': 1072},
 {'

In [ ]:
# COMPLESSITA'

cur = conn.cursor()
cur.execute(f"""
        
    SELECT
        year,
        count(*) as cnt,
        avg(mt) as avg,
        AVG(mt*mt) - AVG(mt)*AVG(mt) as variance
    FROM
        (
        SELECT
            year,
            COALESCE("user", "af user") as matr,
            COALESCE(
                "corso di immatricolazione",
                "corso unico",
                "corso assegnato",
                "corso prenotato",
                "supercorso"
            ) as corso,
            
            /*
                This aggregation is arbitrary. For students applies only one rank the
                results is the same with evrey aggregation. For other, i choose simply take the maximum.
                With avg the results wont change a lot.
            */
            MAX(totale) as mt 
        FROM
            tilstat
        WHERE
            esito LIKE "%IMMATRICOLATO%"
            AND corso LIKE "%AEROSPAZIALE%"
            /*esito NOT LIKE "%RINUNCIA%" and esito NOT LIKE "%ATTESA%"*/
        GROUP BY 
            year, matr
        )
    GROUP BY 
        year

""")
headers = list(map(lambda x: x[0], cur.description))
out = [{h:v for h,v in zip(headers, row)} for row in cur.fetchall()]

# with open('tilstatquart.json', 'w') as outfile:
#     json.dump(out, outfile)
out[:100]

In [ ]:
# {'year': 2022,
#   'cnt': 9227,
#   'avg': 48.51357149920639,
#   'variance': 348.15130310044333}]

In [ ]:
# COMPLESSITA'

cur = conn.cursor()
cur.execute(f"""
        
    SELECT
        year,
        count(*) as cnt,
        avg(totale) as mean,
        (AVG(totale*totale) - AVG(totale)*AVG(totale)) as std,
        (esito LIKE "%IMMATRICOLATO%") as preso
    FROM
    
        (
        SELECT
           COALESCE(
                "corso di immatricolazione",
                "corso unico",
                "corso assegnato",
                "corso prenotato",
                "supercorso"
            ) as corso,
            COALESCE("user", "af user") as matr, year, uri,totale, esito,
            max(grad_n)
        FROM
            tilstat
        WHERE      
           1
        GROUP BY 
            year, corso, matr
        HAVING
            esito NOT LIKE "%RINUNCIA" or 1
        )
    
    group by year
    /*GROUP BY year , preso
    having preso = 1*/

""")
headers = list(map(lambda x: x[0], cur.description))
out = [{h:v for h,v in zip(headers, row)} for row in cur.fetchall()]

# with open('tilstatquart.json', 'w') as outfile:
#     json.dump(out, outfile)
out[:100]

In [ ]:
# # PARANOID CHECKS

# cur = conn.cursor()
# cur.execute("""
#     SELECT
#         date, uri, count(*)
#     FROM tilstat
#     WHERE
#         (
#          "course" == "Ingegneria" AND
         
#          "corso di immatricolazione" IS NULL AND
#          "corso unico" IS NULL AND
#          "corso assegnato" IS NULL AND
#          "corso prenotato" IS NULL AND
#          "corsi in lista di attesa" IS NULL AND
         
#          "esito" NOT LIKE "NON AMMESSO%" AND
#          "esito" NOT LIKE "RINUNCIA ALL'IMMATRICOLAZIONE%" AND
#          "esito" NOT LIKE "RINUNCIA ALLA LISTA DI ATTESA%" AND 
#          "esito" NOT LIKE "RINUNCIA DOPO IMMA%"
#          )
#     GROUP BY date
    
# """)
# list(map(lambda x: x[0], cur.description)), cur.fetchall()

In [ ]:
cur = conn.cursor()
cur.execute("""
    SELECT * FROM tilstat 
""")
list(map(lambda x: x[0], cur.description))

In [ ]:
#















































